# Ejemplo Web scrapping: Mobiliaria Casa Sapo


En este notebook vamos a ver algunos ejemplos para hacer web scrapping, que consiste en obtener cierta información publicada en alguna web. Para ello, nos basaremos en cómo está compuesta la web de la que queremos extraer información.

Todas las webs están diseñadas en HTML*, un lenguaje basado en etiquetas. No nos vamos a meter a conocer el lenguaje HTML aquí, lo único que nos importa de momento es saber que estos códigos son unas cuántas líneas de texto plano con diferentes elementos organizados con etiquetas. Con ellas se define si en una posición va un texto (y su formato), una imagen (que será un enlace a su dirección), un link que te lleve a otra dirección...

Con el *web scrapping* podremos atacar la parte diseñada en HTML. Básicamente, nuestra función será ir mirando las etquetas y quedándonos con aquellas que queramos. Para ello, tendremos que investigar cómo está hecha la web, definir qué queremos "scrapear" e identificar el patrón de etiquetas bajo el que se encuentra. Cabe destacar, que el sistema de etiquetas es un sistema de árbol, donde un elemento identificado por una etiqueta podría contener otros elementos que tuvieran a su vez otras etiquetas.


\**Pese a que todos los sitios web estén basadas en HTML, casi siempre convive junto a otros scripts en otros lenguajes para darle la inteligencia a su web, como podrían ser scripts de Javascript o php. Básicamente, podemos interpretar que el HTML lleva la información y la estructura de la página, mientras que el resto de elementos que aportan funcionalidad a la web serán scripts complementarios en otros lenguajes, como un reproductor de vídeo (con HTML podré ubicarlo en la web pero la funcionalidad quedará de parte de otro lenguaje).*

## Importando librerías

Para obtener el HTML de una web, hay que saber gestionar lo protocolos de transferencia de datos, pero nosotros, al igual que el programador medio, no sabemos hacaerlo. ¿Cómo vamos a hacerlo entonces? Simple, utiliaremos librerías que se encarguen de ello por nosostros. En este caso, nos basaremos en ``requests`` para las conexiones con las webs, y en ``BeautifulSoup`` para analizar los HTML que extraigamos, de modo que nos sea mucho más sencillo detectar las etiquetas y navegar por ellas.

In [33]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

En primer lugar, comenzamos definiendo una cabecera para el intercambio de información, es decir, para simular que somos un navegador y que podamos obtener el texto plano que define el HTML. Lo que añadamos en la cabecera lo vamos a mantener así y no vamos a meternos a ello, pero básicamente es para lo que acabamos de coemntar.

La solicitud de cabecera del Agente de Usuario contiene una cadena característica que permite identificar el protocolo de red que ayuda a descubrir  el tipo de aplicación, sistema operativo, provedor del software o la versión del software de la petición del agente de usuario.

In [34]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

Tras ello, definimos qué web queremos "scrapear" y pedimos su HTML (función ``get``) como si fuéramos un navegador web.

In [35]:
sapo = "https://casa.sapo.pt/comprar-apartamentos/cascais/"

response = get(sapo, headers = headers)

Con esta petición, obtendremos un objeto que hemos guardado en la variable response. Este objeto tiene una estructura diseñada específicamente para la conexión web, y si hacemos un print a lo bestia de él, nos devolverá un código.

Si obtenemos un ``200`` significa que todo ha ido bien, y que el objeto tendrña la información que he solicitado. Sin embargo, puede que devuelva otro código, en cuyo caso hay algo que debemos corregir, no tenemos acceso, hemos introducido mlas la dirección...

In [36]:
print(response) # un 200 es una buena señal :D

<Response [200]>


Como hemos obtenido un 200, significa que todo ha ido bien y que podremos accedera a las diferentes características del mismo, como el contenido de la web, las cabeceras...

A continuación, se muestra lo que se obtiene si hacemos un print de los primeros 1000 caracteres de texto (es el código HTML de la web que estamos atacando):

In [37]:
print(response.text[:1000])



<!DOCTYPE html>

<html lang="pt">
<head><title>
	Casas para Venda, Apartamentos em Cascais, CASA SAPO - Portal Nacional de Imobiliário
</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />
    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="16/01/2021 06:39" />
    
<meta name="content-language" content="pt" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta name="description" content="Casas para Venda, 2929 Apartamentos em Cascais, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />
<meta name="keywords" content="Casas para Venda, Apartamentos em Cascais, venda, compra, comprar, casas, imóveis, apartamentos, moradias, terrenos, porto, lisboa" />
<meta name="referrer" content="always" />
<link rel="alternate" t


Sabemos que las etiquetas se ponen al comienzo y al finalizar lo que va dentro de ella, y tienen una estructura como:
```
<etiqueta propiedad_etiqueta1=propiedad_1 propiedad_etiqueta2=propiedad2 ...> Cosas que van dentro de la etiqueta </etiqueta>
```

Dentro de una etiqueta puede haber otras etiquetas, ya que se sigue una estructura de árbol. Por ejemplo, la etiqueta principal que diferencia las cabeceras (información de configuración no relevante para nuestro análisis) del cuerpo de la web es ``<body></body>``, y dentro de ella estarán el resto.


Ponernos a analizar directamente este texto podría costarnos bastante, por lo que existen librerías que ya lo hacen por nosotros, como ``BeautifulSoup``. Para ello, nos creamos un objeto de esta librería a partir del texto de lo que hemos leído con ``requests``:

In [38]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Ahora que tenemos el obejto, tenemos que buscar las etiquetas que nos interesen, que BeautifulSoup se encargará de buscarlas por nosotros. La sintaxis se muestra a continuación con un ejemplo.

Por ejemplo, si queremos extraer todas las etiquetas ``div`` con el atributo ``class`` igual a ``"searchResultProperty"`` (el cual será una combinación etiqueta + valor de atributo característica para lo que quiera buscar, porque ya hemos mirado el código de la web antes y hemos identificado ese patrón, que en este caso sería de cada casa), la sintaxis sería como sigue:

In [39]:
house_containers = html_soup.find_all('div', class_= "searchResultProperty")

``findall()`` busca *TODAS* las coincidencias y devuelve una lista de strings con cada una de ellas. Veamos qué tenemos en la primera que nos devuelve:

In [41]:
house_containers[0]

<div class="searchResultProperty item G3Position G3Visual hastitle" data-pf="65" data-pkc="false" data-uid="3de33e9d-1938-11eb-8be4-060000000052">
<script type="application/ld+json">{"@context":"http://schema.org","@type":"Offer","image":"http://media.casasapo.pt/Z640x480/Wnone/S5/C2575/P20115915/Tphoto/IDcbf13201-0000-0500-0000-00000d9d48e8.jpg","name":"Apartamento T1 Cascais e Estoril, Cascais","category":"Apartamentos","description":"Apartamento T1 com jardim privativo inserido em condomínio de luxo situado na Gandarinha em Cascais. Com uma localização privilegiada a 200 metros do passadiço que liga o centro de Cascais ao Guincho.  Este imóvel com 76 m2, (...)","price":["760 000 €"],"priceCurrency":["€"],"availableAtOrFrom":{"@type":"Place","address":{"addressCountry":"PT","addressLocality":"Cascais","addressRegion":"Cascais e Estoril"},"geo":{"@type":"GeoCoordinates","latitude":38.6968818429,"longitude":-9.4314301014}},"seller":{"@context":"http://schema.org","@type":"RealEstateAge

Como puedes observar, tal como hemos comentado con anterioridad, tenemos otras etiquetas dentro de nuestras etiquetas. Puede que lo que busquemos esté en un nivel más profundo, por lo que podemos hacer más "zoom" volviendo a preguntar y quedános con lo que realmente nos interesa de ese subconjunto de datos de la web:

In [67]:
first = house_containers[0]
first.find_all("span")

[<span class="titleG3">Apartamento T1 com jardim privativo situado na Gandarinha, Cascais</span>,
 <span>
                         Apartamento T1, Cascais e Estoril, Lisboa
                     </span>,
 <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('3de33e9d-1938-11eb-8be4-060000000052', '13', '5', true, true, '694269'); return false;" style="z-index: 9999;" title="Contacte Anunciante">Contacte Anunciante</span>,
 <span>760 000 <strong title="Euro">€</strong></span>]

El objeto ``first`` ahora tiene la estructura de nuestra primera casa, sacada de ``house_containers``.

Bueno, pues ahora que ya sabemos cómo manejarnos con BeautifulSoup, vamos a proceder a obtener algo de información que nos interese, como el precio de la vivienda o sus características.

### Obteniendo el precio de la vivienda

Tras analizar la web, hemos detectado que, dentro del objeto que hemos etraído en el apartado anterior, podemos identificar el precio de la vivienda en el cuarto objeto con la etiqueta ``span``:

In [68]:
var_1 = first.find_all("span")[3].text
var_1

'760\xa0000 €'

In [69]:
var_1 = var_1.replace('\xa0', '')
var_1

'760000 €'

In [70]:
# Y si solo queremos el número:
# Sacamos caracter a caracter si es número:
var_precio_char = [char for char in var_1 if char.isdigit()]
# Los juntamos en un string (porque tendremos una lista del ejercicio anterior):
var_precio_str = ''.join(var_precio_char)
# Y lo convertimos a entero:
var_precio_int = int(var_precio_str)

In [71]:
print(var_precio_int, type(var_precio_int))

760000 <class 'int'>


### Obteniendo las características de la casa

Del mismo modo que hemos visto lo anterior, podemos extraer otras características de la casa (en base a otros patrones).

In [72]:
# Location
location = first.find_all('p', class_ = "searchPropertyLocation")[0].text
location

'\r\n                    Cascais e Estoril, Lisboa\r\n                '

In [73]:
# Con strip eliminamos todas esas cosas raras a izquierda y derecha de nuestro string:
location.strip()

'Cascais e Estoril, Lisboa'

In [74]:
# Tamaño en m2 (en este caso no está informado):
first.find_all('p')[7]

<p>-</p>

In [75]:
#Obtener fecha de publicación del anuncio
first.find_all('div', class_="searchPropertyDate")

[]

In [55]:
#Fecha del post
pd.to_datetime(first.find_all('div', class_="searchPropertyDate")[0].text[3:10])

IndexError: list index out of range

In [76]:
# Descripción corta:
first.find_all('p', class_="searchPropertyDescription")[0].text[22:-25]

'Apartamento T1 com jardim privativo inserido em condomínio de luxo situado na Gandarinha em Cascais. Com uma localização privilegiada a 200 metros do passadiço que liga o centro de Cascais ao Guincho.  Este imóvel com 76 m2'

Estos ejemplos deberían ser suficientes para que hagamos nuestra propia investigación. El *modus operandi* se reducirá a probar con la estructura html y manipular los valores que se devuelven hasta que obtenemos lo que queremos.

### Continuamos cogiendo todos los links

In [77]:
# Para coger todos los links
for url in first.find_all('a'):
    print(url.get('href'))

https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=694269,694268&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t1-cascais-e-estoril-3de33e9d-1938-11eb-8be4-060000000052.html%3fg3pid%3d694269
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=694269,694268&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t1-cascais-e-estoril-3de33e9d-1938-11eb-8be4-060000000052.html%3fg3pid%3d694269
/agencia/quintela-e-penalva-real-estate/?cl=11244&sys=5
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=694269,694268&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t1-cascais-e-estoril-3de33e9d-1938-11eb-8be4-060000000052.html%3fg3pid%3d694269
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=694269,694268&c=1&MCA=1,1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t1-cascais-e-estoril-3de33e9d-1938-11eb-8be4-060000000052.html%3fg3pid%3d694269


In [20]:
#Obtener todos los links
#first.find_all('a')[0].get('href')[0:-6]

In [78]:
# Creamos una lista que va a formar a nuestro dataframe
titles = []
created = []
prices = []
areas = []
zone = []
condition = []
descriptions = []
urls = []

In [90]:


n_pages = 0

for page in range (0,10):
    n_pages +=1
    sapo_url = "https://casa.sapo.pt/comprar-apartamentos/cascais/"
    r = get(sapo_url, headers = headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_ = "searchResultProperty")
    
    if house_containers != []:
        for container in house_containers:
            try:
                #Precio 
                price = container.find_all('span')[4].text
                print(price)
                if price == 'Contacte Anunciante':
                    price = container.find_all('span')[3].text

                    if price.find('/') != -1:
                        price = price[0:price.find('/')-1]

                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
                print(price)    
                price_ = [int(price[s]) for s in range(0, len(price)) if price[s].isdigit()]

                price = ''
                print(price_)
                for x in price_:
                    price = price + str(x)
                prices.append(int(price))
            
            except:
                continue
            
            #Zona
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[22:location.find(',')]
            zone.append(location)
            
            #Title
            name = container.find_all('span')[0].text
            titles.append(name)
            
            #Description
            desc = first.find_all('p', class_="searchPropertyDescription")[0].text[22:-25]
            descriptions.append(desc)
            
            #Url
            link = container.find_all('a')[0].get('href')[0:-6]
            urls.append(link)
            
    else:
        break
    
    # Lo que estamos haciendo iteración a iteración es leer página a página, y dentro de cada página, nos quedamos con las características 
    #de las que nos interesan. En cada llamada estamos simulando un navegador, por lo que hacerlas muy de seguido podría hacer saltar las alarmas
    #y que sospecharan que somos bots o algo por el estilo, dando lugar a errores de respuesta. Para solucionar esto, se puede poner un 'sleep',
    #que lo que hará será esperar un poquito para enviar la sigueinte instrucción. Aunque en este caso no será necesario:
    #sleep(randint(1,2))
    
print('Hemos \'scrapeado\' {} páginas que contienen un total de {} propiedades.'.format(n_pages, len(titles)))   

Contacte Anunciante

                        Apartamento T3, Cascais e Estoril, Lisboa
                    
[3]
Contacte Anunciante

                        Apartamento T2, Rebelva, São Domingos de Rana, Cascais, Lisboa
                    
[2]
Contacte Anunciante

                        Apartamento T3, Quinta das Marianas (Parede), Carcavelos e Parede, Cascais, Lisboa
                    
[3]
Contacte Anunciante

                        Apartamento T3, Quinta das Marianas (Parede), Carcavelos e Parede, Cascais, Lisboa
                    
[3]
Contacte Anunciante

                        Apartamento T3, Quinta das Marianas (Parede), Carcavelos e Parede, Cascais, Lisboa
                    
[3]
520 000 €
520 000 €
[5, 2, 0, 0, 0, 0]
390 000 €
390 000 €
[3, 9, 0, 0, 0, 0]
995 000 €
995 000 €
[9, 9, 5, 0, 0, 0]
1 200 000 €
1 200 000 €
[1, 2, 0, 0, 0, 0, 0]
725 000 €
725 000 €
[7, 2, 5, 0, 0, 0]
780 000 €
780 000 €
[7, 8, 0, 0, 0, 0]
Contacte Anunciante

                        Apartament

In [96]:
cols = ['Title', 'Price', 'Zone',  'Description', 'URL']

portucasas = pd.DataFrame({'Title': titles,
                          'Price': prices,
                          'Zone': zone,
                          'Description': descriptions,
                          'URL': urls})[cols]

portucasas.to_excel('portucasas.xls', index=False)
portucasas_df = pd.read_excel('portucasas.xls')

In [97]:
portucasas_df

,Title,Price,Zone,Description,URL
0,T2 em condomínio privado,520000,Bairro de Santa Teresinha (Parede),Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
1,T2 renovado no Estoril,390000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
2,Apartamento T2 em condomínio no Estoril,995000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
3,Elegante apartamento T3 no Monte Estoril,1200000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
4,"T3 com áreas amplas, na Costa da Guia",725000,Costa da Guia (Cascais),Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
...,...,...,...,...,...
385,T2 renovado no Estoril,390000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
386,Apartamento T2 em condomínio no Estoril,995000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
387,Elegante apartamento T3 no Monte Estoril,1200000,Cascais e Estoril,Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
388,"T3 com áreas amplas, na Costa da Guia",725000,Costa da Guia (Cascais),Apartamento T1 com jardim privativo inserido e...,https://gespub.casa.sapo.pt/v3/webinterface/cl...


### EJERCICIO

Repasa estos conceptos poniéndolos en práctica. Busca una web en la que veas la información distribuida de una forma que entiendas y puedas obtener patrones, y saca de ellos información que te ayude en un posterior estudio